In [1]:
import datadotworld as dw
import pandas as pd
import psycopg2
import psycopg2.extras as extras
from psycopg2.extras import execute_values

In [2]:
DB_NAME = "ai_product_catalog"
DB_USER = "ai_product_catalog"
DB_PASSWORD = "ai_product_catalog123"
DB_HOST = "127.0.0.1"
DB_PORT = "5432"

CATEGORY_DESC = "Shoes"

In [3]:
product_dataset = dw.load_dataset('data-hut/product-data-from-nike')
product_dataset.describe()


{'name': 'data-hut_product-data-from-nike',
 'title': 'Product data from Nike',
 'description': 'Product data from Nike to help companies and researchers conduct analysis on pricing, product, and assortment.\n\n---\n{"editor":"markdown"}\n---\n\n\nProduct data from Nike from helps companies conduct competition analysis , pricing analyisis . Contact Datahut ( [https://datahut.co/](https://datahut.co/) ) for more details.\n\n\u200b\n\nHere is the link to the competitive analysis we conducted using this data set.\n\n\n@(https://blog.datahut.co/competitive-analysis-nike-vs-adidas/)\n\n\n\u200b',
 'homepage': 'https://data.world/data-hut/product-data-from-nike',
 'license': 'Public Domain',
 'resources': [{'name': 'nike_2020_04_13',
   'path': 'data/nike_2020_04_13.csv',
   'format': 'csv',
   'profile': 'data-resource'},
  {'name': 'original/nike_2020_04_13.csv',
   'path': 'original/nike_2020_04_13.csv',
   'format': 'csv',
   'mediatype': 'text/csv',
   'bytes': 791480,
   'profile': 'da

In [4]:
df = product_dataset.dataframes["nike_2020_04_13"]
df.head()


/opt/anaconda3/envs/ai-product-catalog/lib/python3.12/site-packages/datadotworld/models/dataset.py:198: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pandas.read_csv(


,url,product_name,product_id,listing_price,sale_price,discount,brand,description,rating,reviews,images,last_visited
0,https://www.nike.com/in/t/air-force-1-07-essen...,Nike Air Force 1 '07 Essential,CJ1646-600,0,7495,0,Nike,Let your shoe game shimmer in the Nike Air For...,0.0,0,"[""https://static.nike.com/a/images/t_PDP_1728_...",2020-04-13 15:27:56
1,https://www.nike.com/in/t/air-force-1-07-shoe-...,Nike Air Force 1 '07,CT4328-101,0,7495,0,Nike,The legend lives on in the Nike Air Force 1 '0...,0.0,0,"[""https://static.nike.com/a/images/t_PDP_1728_...",2020-04-13 15:27:56
2,https://www.nike.com/in/t/air-force-1-sage-low...,Nike Air Force 1 Sage Low LX,CI3482-200,0,9995,0,Nike,"Taking both height and craft to new levels, th...",0.0,0,"[""https://static.nike.com/a/images/t_PDP_1728_...",2020-04-13 15:27:56
3,https://www.nike.com/in/t/air-max-dia-se-shoe-...,Nike Air Max Dia SE,CD0479-200,0,9995,0,Nike,"Designed for a woman's foot, the Nike Air Max ...",0.0,0,"[""https://static.nike.com/a/images/t_PDP_1728_...",2020-04-13 15:27:57
4,https://www.nike.com/in/t/air-max-verona-shoe-...,Nike Air Max Verona,CZ6156-101,0,9995,0,Nike,Pass on the good vibes in the Nike Air Max Ver...,0.0,0,"[""https://static.nike.com/a/images/t_PDP_1728_...",2020-04-13 15:27:57


In [5]:
print ("Shape = " + str(df.shape))
print ("Number of unique Product IDs = " + str(len(df['product_id'].drop_duplicates())))
print ("Maximum Length of Product ID Column = " + str(df['product_id'].str.len().max()))
print ("Number of unique Brands = " + str(len(df['brand'].drop_duplicates())))
print ("Maximum Length of Product Name Column = " + str(df['product_name'].str.len().max()))
print ("Maximum Length of Product Description Column = " + str(df['description'].str.len().max()))
print ("Maximum Length of Brand Column = " + str(df['brand'].str.len().max()))


Shape = (643, 12)
Number of unique Product IDs = 554
Maximum Length of Product ID Column = 10
Number of unique Brands = 1
Maximum Length of Product Name Column = 61
Maximum Length of Product Description Column = 532.0
Maximum Length of Brand Column = 4


In [6]:
sql = """
        INSERT INTO categories(category_desc) VALUES (%s) RETURNING category_id;
      """

category_id = None

with psycopg2.connect(database=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT) as db_connection:
  with db_connection.cursor() as c:
      c.execute(sql, (CATEGORY_DESC,))

      rows = c.fetchone()
      if rows:
          category_id = rows[0]

      db_connection.commit()

print("Category Created.  Category_ID = " + str(category_id) + " for Category_Name = " + CATEGORY_DESC)

Category Created.  Category_ID = 1 for Category_Name = Shoes


In [7]:
brandsDF = pd.DataFrame({ "Brand": df['brand'].drop_duplicates() })
print(brandsDF.shape)
brandsDF.head()

(1, 1)


,Brand
0,Nike


In [8]:
with psycopg2.connect(database=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT) as db_connection:
  with db_connection.cursor() as c:
      execute_values(
          cur = c,
          sql = """
                  INSERT INTO brands(brand_desc) VALUES %s
                """,
          argslist = brandsDF.to_dict(orient="records"),
          template = """
                          ( %(Brand)s )
                    """
      )

In [9]:
df['MRPFloat'] = df['sale_price'].astype('float') / 100.0
df['MRPFloat']

0       74.95
1       74.95
2       99.95
3       99.95
4       99.95
        ...  
638    127.97
639     34.97
640     59.47
641    169.95
642     62.97
Name: MRPFloat, Length: 643, dtype: float64

In [10]:
with psycopg2.connect(database=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT) as db_connection:
  with db_connection.cursor() as c:
      execute_values(
          cur = c,
          sql = """
                  INSERT INTO products
                  (
                    sku, 
                    brand_id,
                    product_name,
                    product_desc,
                    size,
                    msrp,
                    category_id
                  ) VALUES %s
                """,
          argslist = df.to_dict(orient="records"),
          template = f"""
                        (
                            %(product_id)s,
                            (select brand_id from brands where brand_desc=%(brand)s),
                            %(product_name)s,
                            %(description)s,
                            null,
                            cast(%(MRPFloat)s as double precision),
                            {category_id}
                        )
                    """
      )